### Automated Skew Reduction


Example usage of an automated skew reduction function I have created.
This will use 3 common methods for reducing skew, mentioned here https://towardsdatascience.com/top-3-methods-for-handling-skewed-data-1334e0debf45
log, sqrt and the boxcox method. 
The function works by finding the optimal minimizer of skew for each column, and transoforms that column with the most optimal function, and then returns the dataframe with the transformed columns.

**Note:** 
<ul>
    <li>A column can not be transformed by the sqrt or boxcox function if any 0 values in that column</li>
    <li>A column can not be transformed by sqrt or boxcox if any negative valus in that column (log accepts negative values)</li>
</ul>

In [161]:
def minimize_skew_with_transform(df, np, stats, SKEW_CUT_OFF=[-1,1]):
    '''
    Finds columns with excess skew and minimizes them the best way possible by checking the best transform to use
    Transforms include: log, sqrt, boxcox
    
    df: dataframe to have transforms
    stats: stats from scipy needed for boxcox transform
    np: numpy reference for func
    
    General rull of thumb for skew cut off value us [-1, 1] but can be changed
    '''
    
    skews_df = pd.DataFrame(df.skew())
    columns_with_high_skew = list()
    
    #get list of skewed data
    for col in skews_df.T.columns:
        skew_score = skews_df.T[col].values[0]
        if skew_score < SKEW_CUT_OFF[0] or skew_score > SKEW_CUT_OFF[1]:
            columns_with_high_skew.append(col)
    
    print('High skew columns: {}\n'.format(', '.join(columns_with_high_skew)))
    
    #find best transform for column and apply it to original df
    for col in columns_with_high_skew:
        
        col_skew = skews_df.T[col].values[0]
        l_trans = 0
        sqrt_trans = 0
        box_cox_trans = 0
        NO_ZEROS = False
        if 0 not in df[col].values:
            l_trans = np.log(df[col]).skew()
            NO_ZEROS = True
        if np.min(df[col]) > 0:
            sqrt_trans = np.sqrt(df[col]).skew()
            if NO_ZEROS:
                box_cox_trans = pd.Series(stats.boxcox(df[col])[0]).skew()
       
        
        #order important for next case statement
        max_vals = [l_trans, sqrt_trans, box_cox_trans, col_skew]
        
        ind = np.argmin(max_vals)
        if ind == 0:
            print('Log transformed: {}'.format(col))
            df[col] = np.log(df[col])
        elif ind == 1:
            print('Sqrt transformed: {}'.format(col))
            df[col] == np.sqrt(df[col])
        elif ind == 2:
            print('Boxcox transformed: {}'.format(col))
            df[col] == pd.Series(stats.boxcox(df[col])[0])
        else:
            #no change as the original value is least skew when unchanged
            pass
            
        
    return df
        

To test this I will use the https://www.kaggle.com/c/house-prices-advanced-regression-techniques dataset


In [162]:
from scipy import stats
import numpy as np
import pandas as pd
train = pd.read_csv("train.csv")

In [154]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [155]:
train.isnull().sum().value_counts()

0       62
81       5
37       3
38       2
8        2
1406     1
1369     1
690      1
1453     1
1179     1
259      1
1        1
dtype: int64

I have shown that there are plenty of null values etc. and that I am not going to change these or any 0's / missing values etc. within the data. However, it will make less changes if the data is not prepared appropriately as the transforms won't be possible if 0's exist or neg values and those columns wil simply be skipped if no transforms are possible.

We can see below the skew in the data set and get an idea of the ones that might be changed (if possible with thier values).

In [156]:
train.skew()

Id                0.000000
MSSubClass        1.407657
LotFrontage       2.163569
LotArea          12.207688
OverallQual       0.216944
OverallCond       0.693067
YearBuilt        -0.613461
YearRemodAdd     -0.503562
MasVnrArea        2.669084
BsmtFinSF1        1.685503
BsmtFinSF2        4.255261
BsmtUnfSF         0.920268
TotalBsmtSF       1.524255
1stFlrSF          1.376757
2ndFlrSF          0.813030
LowQualFinSF      9.011341
GrLivArea         1.366560
BsmtFullBath      0.596067
BsmtHalfBath      4.103403
FullBath          0.036562
HalfBath          0.675897
BedroomAbvGr      0.211790
KitchenAbvGr      4.488397
TotRmsAbvGrd      0.676341
Fireplaces        0.649565
GarageYrBlt      -0.649415
GarageCars       -0.342549
GarageArea        0.179981
WoodDeckSF        1.541376
OpenPorchSF       2.364342
EnclosedPorch     3.089872
3SsnPorch        10.304342
ScreenPorch       4.122214
PoolArea         14.828374
MiscVal          24.476794
MoSold            0.212053
YrSold            0.096269
S

Below I will get the skew of the dataset before and after the changes, and take the mean of each skew to compare the changes. Note the output will outline the rows transformed and the function used.

In [163]:
skew1 = train.skew()
df = minimize_skew_with_transform(train.copy(), np, stats)
skew2 = df.skew()

High skew columns: MSSubClass, LotFrontage, LotArea, MasVnrArea, BsmtFinSF1, BsmtFinSF2, TotalBsmtSF, 1stFlrSF, LowQualFinSF, GrLivArea, BsmtHalfBath, KitchenAbvGr, WoodDeckSF, OpenPorchSF, EnclosedPorch, 3SsnPorch, ScreenPorch, PoolArea, MiscVal, SalePrice

Boxcox transformed: MSSubClass
Log transformed: LotFrontage
Log transformed: LotArea
Log transformed: MasVnrArea
Log transformed: BsmtFinSF1
Log transformed: BsmtFinSF2
Log transformed: TotalBsmtSF
Boxcox transformed: 1stFlrSF
Log transformed: LowQualFinSF
Log transformed: GrLivArea
Log transformed: BsmtHalfBath
Log transformed: KitchenAbvGr
Log transformed: WoodDeckSF
Log transformed: OpenPorchSF
Log transformed: EnclosedPorch
Log transformed: 3SsnPorch
Log transformed: ScreenPorch
Log transformed: PoolArea
Log transformed: MiscVal
Boxcox transformed: SalePrice


In [164]:

print(np.mean(skew1))
print(np.mean(skew2))

2.9668029017648805
0.3178331333636269


Above we can see the strong reduction in average skew, I have arbitrarily picked the range [1, -1] as the measure of exces skew (that is anything outside this range). As I have read this as a general rule of thumb. But this can be changed to have a more strict measure if you desired, by setting the SKEW_CUT_OFF=[new_min, new_max].